In [1]:
import random, copy, csv
from IPython.display import clear_output

In [2]:
class Blackjack():
    def __init__(self, run_status=True, deck=[], players=[], fields=[], num_decks=1):
        self.run_status = run_status
        self.deck = deck
        self.players = players
        self.num_decks = num_decks
        self.fields = ['name', 'password', 'money', 'debt'] # for csv
    
    def createDeck(self):
        deck = []
        suits = ['Hearts', 'Spades', 'Diamonds', 'Clubs']
        faces = list(range(2,11)) + ['Ace', 'Jack', 'Queen', 'King']
        for suit in suits:
            for face in faces:
                deck.append((face,suit))
        self.deck = deck * self.num_decks
    
    def dealCards(self, num_cards):
        random.shuffle(self.deck)
        cards = []
        for i in range(num_cards):
            cards.append(self.deck.pop())
        return cards
    
    def showCards(self, title, cards, num_cards):
        print(f'\n{title}\'s Cards:')
        for i in range(num_cards):
            print(f'Card {i+1}: {cards[i][0]} of {cards[i][1]}')
        if len(cards) > num_cards:
            print(f'{len(cards) - num_cards} face-down card')
        total = int(self.calcHand(cards[:num_cards]))
        print(f'Total: {total}')
    
    def calcHand(self, cards):
        values = list(map(lambda x: x[0] if isinstance(x[0], int) else (10 if x[0] != 'Ace' else [1,11]), cards))
        aces = [isinstance(i, list) for i in values]
        if sum(aces) > 0:
            exclude_aces = sum(list(filter(lambda x: True if x != [1,11] else False, values)))   
            range_values = [exclude_aces + sum(aces), exclude_aces + sum(aces) + 10]
            if range_values[1] > 21:
                return range_values[0]
            else:
                return range_values[1]
        else:
            return sum(values)
    
    def findPlayer(self, name, password=''):
        with open('blackjack.csv', 'r') as f:
            reader = csv.DictReader(f, fieldnames=self.fields)
            
            for row in reader:
                if row['name'] == name:
                    if row['password'] == password:
                        return row
                    return row['name']
            return
    
    def newPlayer(self, name, password):
        with open('blackjack.csv', 'a') as f:
            writer = csv.writer(f, delimiter=',')
            writer.writerow([name, password, 1000, 0]) # initial money=1000 and debt=0
    
    def updateCSV(self):
        updated = []
        with open('blackjack.csv', 'r') as f:
            reader = csv.DictReader(f, fieldnames=self.fields)
            for row in reader:
                for player in self.players:
                    if row['name'] == player.getName():
                        row['money'] = player.getMoney()
                        row['debt'] = player.getDebt()
                updated.append(row)
                
        with open('blackjack.csv', 'w') as f:
            writer = csv.DictWriter(f, fieldnames=self.fields)
            for row in updated:
                writer.writerow(row)
        
    def startGame(self, dealer):
        self.createDeck()
        num_players = len(self.players)
        while num_players == 0:
            try:
                num_players += int(input('How many players? '))
            except:
                print('Please enter a valid number. The maximum number of players is 7.')
        
        # check for existing players or add new ones. names must be unique.
        while len(self.players) <  num_players: 
            name = input('Enter player name: ').strip().title()
            if self.findPlayer(name) == name:
                password = input('Enter player password: ').strip()
                result = self.findPlayer(name, password)
                if isinstance(result, dict):
                    money = float(result['money'])
                    debt = float(result['debt'])
                    player = Player(name, money, debt)
                    self.players.append(player)
                    print(f'{player.getName()} has ${"{:.2f}".format(money)} and is ${"{:.2f}".format(debt)} in debt.')
                else:
                    print('Incorrect password.')
            else:
                password = input('Enter new player password: ').strip()
                self.newPlayer(name, password)
                player = Player(name)
                self.players.append(player)
   
        # players must buy chips if they have no money. otherwise, they do not play.                  
        for player in self.players:
            player.setHand([])
            print(f'\n{player.getName()} has ${"{:.2f}".format(player.getMoney())}')
            if player.getMoney() == 0:
                add = ''
                while add not in ['y', 'n']:
                    try:
                        add = input(f'{player.getName()}, would you like to buy more chips? (y/n) ').lower().strip()
                        if add == 'y':
                            amount = 0
                            while amount == 0:
                                try:
                                    amount += float(input('How much would you like to buy? '))
                                    player.buyChips(amount)
                                except:
                                    print('Please enter a valid amount.')
                        else:
                            break
                    except:
                        print('Please enter y or n.')
                if add == 'n':
                    continue
            
            bet = 0
            while bet == 0:
                try:
                    bet += float(input(f'{player.getName()}, how much to bet? '))
                except:
                    print('Please enter a valid amount.')
                if bet > player.getMoney():
                    bet = 0
                    print('Insufficient funds.')
            
            player.setBet(bet)
            player.setHand(self.dealCards(2))
            
        dealer.setHand(self.dealCards(2))
        self.showCards(dealer.getName(), dealer.getHand(), 1)
    
    # sets player options and goes through multiple hands, if applicable
    def playerMove(self, player):
        seen_hands = []
        options = ['hit', 'stand']
        if player.getMoney() >= (player.getBet() * 2):
            options.append('double down')
        if len(set([card[0] for card in player.getHand()])) == 1 and len(player.getHand()) == 2:
            options.append('split')
        current_hand = copy.copy(player.getHand())
        while current_hand not in seen_hands:
            self.perHand(player, options)
            if len(current_hand) > len(player.getHand()):
                  seen_hands.append(current_hand)
            else:
                  seen_hands.append(player.getHand())
            player.nextHand()
            current_hand = copy.copy(player.getHand())
    
    def perHand(self, player, options):
        total = self.calcHand(player.getHand())
        self.showCards(player.getName(), player.getHand(), len(player.getHand()))
        while total < 21:
            ask = ['-'.join(options[-2:]).replace('-', ' or ')]
            for o in options[:-2]:
                  ask.append(o + ', ')
            action = input(f'{player.getName()}: {"".join(ask[::-1])}? ').strip().lower()
            if action not in options:
                print('Please choose a valid action.')
            else:
                print(f'{player.getName()} has chosen to {action}.')
                if action in ['hit', 'double down']:
                    player.setHand(player.getHand() + self.dealCards(1))
                    self.showCards(player.getName(), player.getHand(), len(player.getHand()))
                    if action == 'double down':
                        player.setBet(player.getBet() * 2)
                        break
                elif action == 'split':
                    player.splitHand()
                    break
                else:
                    break
            total = self.calcHand(player.getHand())
                           
    def compareTotal(self, player, dealer):
        d = self.calcHand(dealer.getHand())
        p = self.calcHand(player.getHand())
        if p == 21 and len(player.getHand()) == 2:
            player.setBet(player.getBet() * 1.5)
        if p < 22 and d < 22:
            if p == d:
                print(f'\n{player.getName()} and dealer tie.')
            elif p > d:
                print(f'\n{player.getName()}, you win ${"{:.2f}".format(player.getBet())}!')
                player.win()
            else:
                print(f'\n{player.getName()}, you lose ${"{:.2f}".format(player.getBet())}!')
                player.lose()
        elif p > 21:
            print(f'\n{player.getName()}, you went bust! You lose ${"{:.2f}".format(player.getBet())}!')
            player.lose()
        else:
            print(f'\nDealer went bust. {player.getName()}, you win ${"{:.2f}".format(player.getBet())}!')
            player.win()
        print(f'{player.getName()} has ${"{:.2f}".format(player.getMoney())}')
                           
    def play(self):
        playing = input('Play blackjack? (y/n) ').strip().lower()
        while self.run_status:
            if playing == 'n':
                self.run_status = False
                self.updateCSV()
                break
            
            dealer = Dealer()
            self.startGame(dealer)
            for player in self.players:
                if player.getMoney() > 0:
                    self.playerMove(player)
            
            self.showCards(dealer.getName(), dealer.getHand(), len(dealer.getHand()))
            while self.calcHand(dealer.getHand()) < 17:
                dealer.setHand(dealer.getHand() + self.dealCards(1))
                self.showCards(dealer.getName(), dealer.getHand(), len(dealer.getHand()))
            
            for player in self.players:
                if player.getMoney() > 0:
                    seen_hands = []
                    while player.getHand() not in seen_hands:
                        self.compareTotal(player, dealer)
                        seen_hands.append(player.getHand())
                        player.nextHand()
            playing = input('Continue playing? (y/n) ')
            clear_output()

In [3]:
class Gamer():
    def __init__(self, name, hand=[]):
        self.name = name
        self.hand = hand
    
    def getName(self):
        return self.name
    
    def getHand(self): # returns hand, or first hand if there are multiple
        if any([isinstance(h, list) for h in self.hand]):
            return self.hand[0]
        else:
            return self.hand
    
    def setHand(self, new_hand): # modifies hand, or first hand if there are multiple
        if any([isinstance(h, list) for h in self.hand]):
            self.hand[0] = new_hand
        else:
            self.hand = new_hand
            
    def nextHand(self): # shuffles list of hands
        old_hand = self.hand.pop(0)
        self.hand.append(old_hand)
    
    def splitHand(self):
        self.hand[0] = [self.hand[0]]
        self.hand[1] = [self.hand[1]]

class Player(Gamer):
    def __init__(self, name, money=1000, debt=0, bet=0):
        super().__init__(name)
        self.bet = bet
        self.money = money
        self.debt = debt
        
    def buyChips(self, amount):
        self.money += amount
        self.debt += amount
    
    def getMoney(self):
        return self.money
    
    def getDebt(self):
        return self.debt
    
    def getBet(self):
        return self.bet
    
    def setBet(self, bet):
        self.bet = bet

    def win(self):
        self.money += self.bet
    
    def lose(self):
        self.money -= self.bet
        
class Dealer(Gamer):
    def __init__(self, name='Dealer'):
        super().__init__(name)

In [4]:
game = Blackjack()
game.play()